<a href="https://colab.research.google.com/github/victorvalente/EvoMoE/blob/main/EvoMoE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.6 MB/s eta 0:00:00


In [39]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import json
import ast
import io
import random
import re
import os # For checking file existence
import sys # For potential command-line arguments
import itertools
from collections import defaultdict
import math
import traceback # For more detailed error logging if needed

# Import optimization libraries
from deap import base, creator, tools, algorithms
# Ensure scipy.optimize is imported for relevant problems
from scipy.optimize import minimize, Bounds, LinearConstraint, differential_evolution
# Import curve_fit if needed for fitting, though not directly used in basic portfolio opt
# from scipy.optimize import curve_fit

# --- Helper Functions ---

def parse_array_string(arr_str):
    """Safely parses a string representation of a numpy array."""
    if not isinstance(arr_str, str) or not arr_str.startswith('array('): return None
    try:
        processed_str = re.sub(r'\s+', ' ', arr_str.replace('array(', '', 1).strip())
        # Remove trailing ')' if it exists, careful not to remove it from numbers
        if processed_str.endswith(')') and '(' not in processed_str.split(')')[-1]: processed_str = processed_str[:-1]
        # Add handling for potential 'dtype=' argument if present
        processed_str = re.sub(r', dtype=[\w\.]+(\s*|,|\))', r'\1', processed_str).strip() # Handle dtype slightly better
        if processed_str.endswith(')'): processed_str = processed_str[:-1] # Re-check after dtype removal

        list_data = ast.literal_eval(processed_str)
        return np.array(list_data)
    except Exception as e: print(f"Warn: Err parsing array: {e}\nStr: {arr_str[:100]}..."); return None

def parse_problem_data(df, index):
    """Extracts and parses data for a given problem index."""
    if index not in df['ProblemIndex'].values: raise ValueError(f"ProblemIndex {index} not found.")
    problem_row = df.loc[df['ProblemIndex'] == index].iloc[0]
    problem_type = problem_row['DetectedType']; description = problem_row['OriginalDescription']
    data = {'problem_type': problem_type, 'description': description, 'problem_index': index}
    print(f"\n--- Parsing Problem Index: {index}, Type: {problem_type} ---")
    for col, value in problem_row.items():
        if pd.isna(value) or value == "" or col in ['ProblemIndex', 'DetectedType', 'OriginalDescription']: continue
        try:
            original_value = value; parsed = False; temp_value = value # Use temp_value for parsing attempts

            if isinstance(temp_value, str):
                 # --- FIX: Strip potential outer quotes added by CSV writer ---
                 if len(temp_value) > 1 and temp_value.startswith('"') and temp_value.endswith('"'):
                     temp_value = temp_value[1:-1].replace('""', '"') # Remove outer quotes, unescape internal quotes
                 # ------------------------------------------------------------

                 # ... Debug print for Index 1 ...
                 if col == 'list_of_cities' and data.get('problem_index') == 1:
                     print(f"DEBUG(Idx1): Parsing 'list_of_cities'. Post-strip type:{type(temp_value)}, val:{str(temp_value)[:100]}")
                     try: parsed_val_debug = ast.literal_eval(temp_value); print(f"DEBUG(Idx1): Parsed type:{type(parsed_val_debug)}")
                     except Exception as e_debug: print(f"DEBUG(Idx1): Failed parse: {e_debug}")

                 # Try parsing complex types using the potentially stripped string
                 if (temp_value.startswith('{') and temp_value.endswith('}')) or \
                    (temp_value.startswith('[') and temp_value.endswith(']')):
                     try: data[col] = json.loads(temp_value); parsed = True
                     except json.JSONDecodeError:
                          try:
                              if "__" in temp_value: print(f"Warn: Skip ast on unsafe str '{col}'.")
                              else: data[col] = ast.literal_eval(temp_value); parsed = True
                          except: pass # Keep parsing attempt order

                 if not parsed and temp_value.startswith('array('):
                     # Pass the potentially stripped string to array parser
                     parsed_array = parse_array_string(temp_value)
                     if parsed_array is not None: data[col] = parsed_array; parsed = True

                 # Try numeric if not parsed
                 numeric_cols = ['investment_amount','num_addresses_expected','num_drivers','num_homes','num_new_shops',
                                 'num_nurses','num_rooms','num_sessions','num_shifts','vehicle_mpg','min_distance_miles',
                                 'building_stories','max_consecutive_days','minimum_pressure_requirements_at_nodes','num_days']
                 if not parsed and col in numeric_cols:
                      # Use original value for numeric conversion if stripping quotes wasn't intended? No, use temp_value.
                      val_to_convert = temp_value # Use the potentially stripped value
                      try: data[col] = float(val_to_convert); parsed = True
                      except ValueError:
                           try: data[col] = int(val_to_convert); parsed = True
                           except ValueError: pass # Keep original value if not parsed

                 # Default: store original string if no parsing succeeded
                 if not parsed: data[col] = original_value
            else:
                 data[col] = value # Value wasn't a string initially
        except Exception as e: print(f"Warn: Unhandled parse err '{col}'. Val:'{str(value)[:100]}...'. Err:{e}. Storing str."); data[col] = original_value

    # Final check for critical fields like num_days
    if index == 5 and not isinstance(data.get('num_days'), (int, float)):
        print(f"Parser Check: num_days for Index 5 is '{data.get('num_days')}' after parsing.")
        data['num_days'] = None
    return data


# --- Objective/Evaluation Functions (remain the same) ---
def evaluate_tsp_route(individual, matrix):
    distance = 0; n = len(individual)
    for i in range(n): distance += matrix[individual[i]][individual[(i + 1) % n]]
    return distance
def evaluate_knapsack(individual, items_data, capacity):
    total_weight, total_value = 0.0, 0.0
    for i, item in enumerate(items_data):
        if individual[i] == 1: total_weight += item['weight']; total_value += item['value']
    return (total_value,) if total_weight <= capacity else (0,)
def evaluate_vrp_distance(individual, dist_matrix, num_drivers, depot=0):
    total_distance = 0; n_customers = len(individual)
    if num_drivers <= 0 : return float('inf'),
    customers_per_driver = math.ceil(n_customers / num_drivers)
    start_cust_idx = 0; customer_indices = individual[:]
    for _ in range(num_drivers):
        current_route = [depot]; route_customers = customer_indices[start_cust_idx : start_cust_idx + customers_per_driver]
        current_route.extend(route_customers); current_route.append(depot); start_cust_idx += len(route_customers)
        route_dist = 0
        for i in range(len(current_route) - 1):
             from_idx, to_idx = current_route[i], current_route[i+1]
             if 0 <= from_idx < dist_matrix.shape[0] and 0 <= to_idx < dist_matrix.shape[1]: route_dist += dist_matrix[from_idx][to_idx]
             else: return float('inf'),
        total_distance += route_dist
        if start_cust_idx >= n_customers: break
    return total_distance,
def evaluate_facility_location(individual, target_area_bounds, num_demand_points=100):
    num_facilities = len(individual) // 2
    if num_facilities == 0: return float('inf'),
    facility_coords = np.array(individual).reshape(num_facilities, 2)
    min_x, max_x, min_y, max_y = target_area_bounds
    demand_points = np.random.rand(num_demand_points, 2)
    demand_points[:, 0] = demand_points[:, 0] * (max_x - min_x) + min_x
    demand_points[:, 1] = demand_points[:, 1] * (max_y - min_y) + min_y
    max_dist_overall = 0
    for dem_pt in demand_points:
        distances = np.linalg.norm(facility_coords - dem_pt, axis=1)
        min_dist_to_facility = np.min(distances) if distances.size > 0 else float('inf')
        max_dist_overall = max(max_dist_overall, min_dist_to_facility)
    return max_dist_overall,
def evaluate_nurse_scheduling(individual, num_nurses, num_days, max_consecutive):
    if not individual: return float('inf'),
    try: schedule = np.array(individual).reshape(num_nurses, num_days)
    except ValueError as e: print(f"Err reshape sched: {e}. Len:{len(individual)}, Exp:({num_nurses},{num_days})"); return float('inf'),
    penalty = 0
    for n in range(num_nurses):
        consecutive_count = 0
        for d in range(num_days):
            if schedule[n, d] > 0: consecutive_count += 1
            else:
                if consecutive_count > max_consecutive: penalty += (consecutive_count - max_consecutive) * 10
                consecutive_count = 0
        if consecutive_count > max_consecutive: penalty += (consecutive_count - max_consecutive) * 10
    return penalty,
def portfolio_variance(weights, cov_matrix): return weights @ cov_matrix @ weights
def portfolio_return(weights, expected_returns): return np.sum(weights * expected_returns)
def evaluate_timetabling(individual, sessions, rooms, slots):
    penalty = 0; num_slots, num_sessions, num_rooms = len(slots), len(sessions), len(rooms); room_keys = list(rooms.keys())
    if len(individual) != num_sessions: return float('inf'),
    schedule_by_slot = defaultdict(list); room_assignments = defaultdict(dict); speakers_in_slot = defaultdict(set); unassigned_sessions = []
    for session_idx, slot_idx in enumerate(individual):
        if not (0 <= slot_idx < num_slots): return float('inf'),
        session_info = sessions[session_idx]
        if not isinstance(session_info, dict): return float('inf'),
        required_attendance = session_info.get('attendance', 0); speaker = session_info.get('speaker', f'UnknownS_{session_idx}'); assigned_room = False
        possible_rooms = random.sample(room_keys, num_rooms)
        for room_key in possible_rooms:
            room_capacity = rooms.get(room_key, float('inf'))
            if not isinstance(room_capacity, (int, float)): room_capacity = float('inf')
            if room_key in room_assignments.get(slot_idx, {}): continue
            if required_attendance > room_capacity: continue
            if speaker in speakers_in_slot.get(slot_idx, set()): continue
            schedule_by_slot[slot_idx].append((session_idx, room_key))
            if slot_idx not in room_assignments: room_assignments[slot_idx] = {}
            room_assignments[slot_idx][room_key] = session_idx
            if slot_idx not in speakers_in_slot: speakers_in_slot[slot_idx] = set()
            speakers_in_slot[slot_idx].add(speaker); assigned_room = True; break
        if not assigned_room: unassigned_sessions.append(session_idx)
    penalty += len(unassigned_sessions) * 1000
    speakers_check = defaultdict(set)
    for slot_idx, assignments in schedule_by_slot.items():
        if not isinstance(assignments, list): penalty+=5000; continue
        current_speakers = set()
        for item in assignments:
             if not isinstance(item, tuple) or len(item)!=2: penalty+=5000; continue
             session_idx, room_key = item
             if not (0 <= session_idx < num_sessions): penalty+=5000; continue
             session_info_chk = sessions[session_idx]
             if not isinstance(session_info_chk, dict): penalty+=5000; continue
             speaker_chk = session_info_chk.get('speaker', f'Unknown_{session_idx}')
             if speaker_chk in current_speakers: penalty += 100
             current_speakers.add(speaker_chk)
    return penalty,
def evaluate_project_scheduling(individual, tasks):
    task_map = {task['task_id']: i for i, task in enumerate(tasks)}; task_finish_times = {}; max_finish_time = 0; valid_ids = {task['task_id'] for task in tasks}
    for task_idx in individual:
        task = tasks[task_idx]; task_id = task['task_id']; duration = task['duration_days']; dependencies = task.get('depends_on', []); earliest_start = 0
        for dep_id in dependencies:
             if dep_id not in valid_ids or dep_id not in task_finish_times: return float('inf'),
             earliest_start = max(earliest_start, task_finish_times[dep_id])
        finish_time = earliest_start + duration; task_finish_times[task_id] = finish_time; max_finish_time = max(max_finish_time, finish_time)
    return max_finish_time,

# --- Main Execution Logic ---
def run_optimization(problem_data, problem_index):
    """Sets up and runs optimization, returning a results dictionary."""
    problem_type = problem_data.get('problem_type')
    print(f"\n>>> Processing Problem Index: {problem_index} ({problem_type}) <<<")
    result = { # Initialize default result
        "ProblemIndex": problem_index, "DetectedType": problem_type, "Status": "Failed - Unknown Error",
        "ObjectiveValue": None, "ObjectiveValue2": None, "ResultDetails": "" }

    try:
        # --- Problem-Specific Porting ---
        if problem_type == "TSP_FLIGHTS":
            print("--- Porting TSP_FLIGHTS ---")
            cities=problem_data.get('list_of_cities'); cost_matrix=problem_data.get('flight_cost_matrix'); duration_matrix=problem_data.get('flight_duration_matrix'); transfer_times=problem_data.get('airport_transfer_times_hours')
            if not all([isinstance(cities, list), isinstance(cost_matrix, np.ndarray), isinstance(duration_matrix, np.ndarray), isinstance(transfer_times, list)]): result.update({"Status": "Failed - Missing/Invalid Data", "ResultDetails": "Missing cities, matrices, or transfer times."}); return result
            num_cities = len(cities)
            if cost_matrix.shape!=(num_cities, num_cities) or duration_matrix.shape!=(num_cities, num_cities) or len(transfer_times)!=num_cities: result.update({"Status": "Failed - Data Dimension Mismatch", "ResultDetails": f"Matrices/times dim mismatch for {num_cities} cities."}); return result
            print(f"Data loaded: {num_cities} cities.")
            print("=== DEAP Setup (Multi-Obj GA) ===");
            if hasattr(creator, "FitnessMulti"): del creator.FitnessMulti
            if hasattr(creator, "Individual"): del creator.Individual
            creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0)); creator.create("Individual", list, fitness=creator.FitnessMulti)
            toolbox=base.Toolbox(); toolbox.register("indices", random.sample, range(num_cities), num_cities); toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices); toolbox.register("population", tools.initRepeat, list, toolbox.individual)
            def evaluate_tsp_flights(individual): tc=evaluate_tsp_route(individual, cost_matrix); td=evaluate_tsp_route(individual, duration_matrix); td += sum(transfer_times[idx] for idx in individual); return tc, td
            toolbox.register("evaluate", evaluate_tsp_flights); toolbox.register("mate", tools.cxOrdered); toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05); toolbox.register("select", tools.selNSGA2)
            print("--- Running DEAP NSGA-II ---"); POP_SIZE,MAX_GEN,CXPB,MUTPB = 50,50,0.7,0.2; pop=toolbox.population(n=POP_SIZE); hof=tools.ParetoFront();
            algorithms.eaMuPlusLambda(pop, toolbox, mu=POP_SIZE, lambda_=POP_SIZE, cxpb=CXPB, mutpb=MUTPB, ngen=MAX_GEN, halloffame=hof, verbose=False)
            print("--- DEAP Results ---"); print(f"Non-dom solutions:{len(hof)}")
            if hof: best_sol = hof[0]; fitness = tuple(float(v) for v in best_sol.fitness.values); route_str = ' -> '.join([cities[idx] for idx in best_sol]) + ' -> ' + cities[best_sol[0]]; result.update({"Status": "Success", "ObjectiveValue": fitness[0], "ObjectiveValue2": fitness[1], "ResultDetails": f"Found {len(hof)} non-dom solutions. Ex Route: {route_str}; Fit(Cost,Dur):{fitness}"}); print(f" Sol 1 (Example): {route_str} | Fit(Cost,Dur):{fitness}")
            else: result.update({"Status": "Success - No Solution Found", "ResultDetails": "Algorithm finished but HallOfFame is empty."}); print("No solution found.")

        elif problem_type == "TSP_DRIVING_FUEL":
            print("--- Porting TSP_DRIVING_FUEL ---")
            locations=problem_data.get('list_of_cities'); dist_matrix=problem_data.get('driving_distance_matrix_miles'); mpg=problem_data.get('vehicle_mpg')
            if not isinstance(locations, list): result.update({"Status": "Failed - Missing/Invalid Data", "ResultDetails": f"Missing/invalid 'list_of_cities'. Parsed:{locations}. Check CSV."}); return result
            if not isinstance(dist_matrix, np.ndarray): result.update({"Status": "Failed - Missing/Invalid Data", "ResultDetails": "Missing/invalid distance matrix (needs actual numpy array)."}); return result
            if not isinstance(mpg,(float,int)) or mpg<=0: result.update({"Status": "Failed - Missing/Invalid Data", "ResultDetails": "Invalid 'vehicle_mpg'."}); return result
            num_locations=len(locations);
            if dist_matrix.shape!=(num_locations, num_locations): result.update({"Status": "Failed - Data Dimension Mismatch", "ResultDetails": f"Matrix dim mismatch ({dist_matrix.shape}) for {num_locations} locations."}); return result
            print(f"Data loaded: {num_locations} locations, MPG:{mpg}")
            print("=== DEAP Setup (GA - Min Fuel) ===");
            if hasattr(creator, "FitnessMin"): del creator.FitnessMin
            if hasattr(creator, "Individual"): del creator.Individual
            creator.create("FitnessMin", base.Fitness, weights=(-1.0,)); creator.create("Individual", list, fitness=creator.FitnessMin)
            toolbox=base.Toolbox(); toolbox.register("indices", random.sample, range(num_locations), num_locations); toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices); toolbox.register("population", tools.initRepeat, list, toolbox.individual)
            toolbox.register("evaluate", lambda ind: (evaluate_tsp_route(ind, dist_matrix)/mpg,)); toolbox.register("mate", tools.cxOrdered); toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05); toolbox.register("select", tools.selTournament, tournsize=3)
            print("--- Running DEAP GA ---"); POP_SIZE,MAX_GEN,CXPB,MUTPB = 50,50,0.7,0.2; pop=toolbox.population(n=POP_SIZE); hof=tools.HallOfFame(1);
            algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=MAX_GEN, halloffame=hof, verbose=False)
            print("--- DEAP Results ---")
            if hof: best_ind=hof[0]; route=[locations[idx] for idx in best_ind]; fuel=best_ind.fitness.values[0]; dist=evaluate_tsp_route(best_ind, dist_matrix); route_str = ' -> '.join(route) + ' -> ' + route[0]; result.update({"Status": "Success", "ObjectiveValue": fuel, "ResultDetails": f"Best Route: {route_str} | Fuel:{fuel:.2f}gal, Dist:{dist:.2f}mi"}); print(f"Best Route: {route_str} | Fuel:{fuel:.2f}gal, Dist:{dist:.2f}mi")
            else: result.update({"Status": "Success - No Solution Found", "ResultDetails": "Algorithm finished but HallOfFame is empty."}); print("No solution found.")

        elif problem_type == "KNAPSACK_MOVING":
            print("--- Porting KNAPSACK_MOVING ---")
            items_raw=problem_data.get('item_list_dimensions_values'); truck_dims=problem_data.get('truck_dimensions')
            # Add specific type checks after parsing
            if not isinstance(items_raw, list): result.update({"Status":"Failed - Invalid Data", "ResultDetails":f"Invalid items list. Type:{type(items_raw)}" }); return result
            if not isinstance(truck_dims, list) or len(truck_dims)!=3: result.update({"Status":"Failed - Invalid Data", "ResultDetails":f"Invalid truck dims. Type:{type(truck_dims)}" }); return result
            items_data=[]; required_keys=['name','width_cm','height_cm','depth_cm','value_usd']
            try:
                # Check truck_dims content type before conversion
                if not all(isinstance(d, (int, float, str)) for d in truck_dims):
                     raise TypeError(f"Truck dimensions list contains non-numeric/non-string types: {truck_dims}")
                truck_volume=np.prod([float(d) for d in truck_dims]); print(f"Truck Vol:{truck_volume:.0f}cc"); print("Items:")
                for item_dict in items_raw:
                    if not isinstance(item_dict, dict) or not all(k in item_dict for k in required_keys): print(f"Warn: Skip item {item_dict.get('name','N/A') if isinstance(item_dict, dict) else 'Invalid Format'}"); continue
                    vol=float(item_dict['width_cm'])*float(item_dict['height_cm'])*float(item_dict['depth_cm']); val=float(item_dict['value_usd'])
                    items_data.append({'name':item_dict['name'],'weight':vol,'value':val}); print(f"  - {item_dict['name']}: Vol={vol:.0f}, Val=${val:.2f}")
                if not items_data: result.update({"Status":"Failed - Invalid Data", "ResultDetails":"No valid items."}); return result
                num_items=len(items_data)
            except Exception as e: result.update({"Status":"Failed - Data Processing Error", "ResultDetails":f"Error item/truck data: {e}"}); return result
            print("=== DEAP Setup (GA - Max Value) ===");
            if hasattr(creator, "FitnessMax"): del creator.FitnessMax
            if hasattr(creator, "Individual"): del creator.Individual
            creator.create("FitnessMax", base.Fitness, weights=(1.0,)); creator.create("Individual", list, fitness=creator.FitnessMax)
            toolbox=base.Toolbox(); toolbox.register("attr_bool", random.randint, 0, 1); toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, num_items); toolbox.register("population", tools.initRepeat, list, toolbox.individual)
            toolbox.register("evaluate", evaluate_knapsack, items_data=items_data, capacity=truck_volume); toolbox.register("mate", tools.cxTwoPoint); toolbox.register("mutate", tools.mutFlipBit, indpb=0.05); toolbox.register("select", tools.selTournament, tournsize=3)
            print("--- Running DEAP GA ---"); POP_SIZE,MAX_GEN,CXPB,MUTPB = 50,40,0.7,0.2; pop=toolbox.population(n=POP_SIZE); hof=tools.HallOfFame(1);
            algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=MAX_GEN, halloffame=hof, verbose=False)
            print("--- DEAP Results ---")
            if hof: best_ind=hof[0]; val=best_ind.fitness.values[0]; vol=sum(items_data[i]['weight'] for i,bit in enumerate(best_ind) if bit==1); items_list=[items_data[i]['name'] for i,bit in enumerate(best_ind) if bit==1] or ['None']; result.update({"Status":"Success", "ObjectiveValue":val, "ResultDetails":f"Volume:{vol:.0f}/{truck_volume:.0f}, Items:{items_list}"}); print(f"Best value:${val:.2f}, Vol:{vol:.0f}/{truck_volume:.0f}, Items:{items_list}")
            else: result.update({"Status":"Success - No Solution Found"}); print("No solution found.")

        elif problem_type == "VRP_MANHATTAN":
            print("--- Porting VRP_MANHATTAN (Simplified: Min Distance) ---")
            dist_matrix=problem_data.get('driving_distance_matrix_miles'); num_drivers=problem_data.get('num_drivers'); addresses=problem_data.get('delivery_addresses_list')
            if not isinstance(dist_matrix, np.ndarray): result.update({"Status": "Failed - Missing Data", "ResultDetails":f"Requires 'driving_distance_matrix_miles' (Numpy Array). Got type {type(dist_matrix)}. Calc from addrs not implemented."}); return result
            if not isinstance(num_drivers,(int,float)) or num_drivers<=0: result.update({"Status":"Failed - Invalid Data", "ResultDetails":"Invalid 'num_drivers'."}); return result
            num_drivers=int(num_drivers)
            if not isinstance(addresses, list): result.update({"Status":"Failed - Invalid Data", "ResultDetails":f"Invalid 'delivery_addresses_list'. Type:{type(addresses)}" }); return result
            num_locations=dist_matrix.shape[0]; num_customers=0; addr_list_offset=0
            if num_locations==len(addresses)+1: num_customers=len(addresses); addr_list_offset=1; print(f"Assume matrix includes depot. {num_customers} cust, {num_drivers} drivers.")
            elif num_locations==len(addresses): num_customers=len(addresses)-1; addr_list_offset=1; print(f"Assume matrix idx 0=depot. {num_customers} cust, {num_drivers} drivers.")
            else: result.update({"Status":"Failed - Data Dimension Mismatch", "ResultDetails":f"Matrix/Addr mismatch ({num_locations} vs {len(addresses)})."}); return result
            if num_customers <= 0: result.update({"Status":"Failed - Invalid Data", "ResultDetails":"No customers identified."}); return result
            print("=== DEAP Setup (GA - Min VRP Dist) ===");
            if hasattr(creator, "FitnessMin"): del creator.FitnessMin
            if hasattr(creator, "Individual"): del creator.Individual
            creator.create("FitnessMin", base.Fitness, weights=(-1.0,)); creator.create("Individual", list, fitness=creator.FitnessMin)
            toolbox=base.Toolbox(); customer_indices=list(range(1, num_customers+1)); toolbox.register("indices", random.sample, customer_indices, num_customers); toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices); toolbox.register("population", tools.initRepeat, list, toolbox.individual)
            toolbox.register("evaluate", evaluate_vrp_distance, dist_matrix=dist_matrix, num_drivers=num_drivers, depot=0); toolbox.register("mate", tools.cxPartialyMatched); toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05); toolbox.register("select", tools.selTournament, tournsize=3)
            print("--- Running DEAP GA ---"); POP_SIZE,MAX_GEN,CXPB,MUTPB = 60,60,0.7,0.2; pop=toolbox.population(n=POP_SIZE); hof=tools.HallOfFame(1);
            algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=MAX_GEN, halloffame=hof, verbose=False)
            print("--- DEAP Results (Simplified VRP) ---")
            if hof:
                 best_ind=hof[0]; min_dist=best_ind.fitness.values[0]; print(f"Best Total Dist:{min_dist:.2f}"); result_routes = []
                 customers_per_driver=math.ceil(num_customers/num_drivers); start_cust_idx=0; best_cust_order=best_ind[:]
                 for d in range(num_drivers):
                      route_indices=[0]; route_customers=best_cust_order[start_cust_idx:start_cust_idx+customers_per_driver]; route_indices.extend(route_customers); route_indices.append(0); route_addrs=[]
                      for idx in route_indices:
                           addr_idx = idx - addr_list_offset
                           if idx==0: route_addrs.append("DEPOT_0")
                           elif 0<=addr_idx<len(addresses): route_addrs.append(f"Addr_{idx}({str(addresses[addr_idx])[:15]}...)")
                           else: route_addrs.append(f"INVALID_IDX_{idx}")
                      route_str = f"Driver {d+1}: {' -> '.join(route_addrs)}"
                      result_routes.append(route_str); print(route_str)
                      start_cust_idx+=len(route_customers);
                      if start_cust_idx>=len(best_cust_order): break
                 result.update({"Status":"Success", "ObjectiveValue":min_dist, "ResultDetails": "; ".join(result_routes)})
            else: result.update({"Status":"Success - No Solution Found"}); print("No solution found.")

        elif problem_type == "FACILITY_LOCATION_SEATTLE":
             print("--- Porting FACILITY_LOCATION_SEATTLE (Simplified: Min Max Dist) ---")
             num_shops=problem_data.get('num_new_shops'); min_dist=problem_data.get('min_distance_miles'); target_area=problem_data.get('target_geographic_area')
             if not isinstance(num_shops,(int,float)) or num_shops<=0: result.update({"Status":"Failed - Invalid Data", "ResultDetails":"Invalid 'num_new_shops'."}); return result
             num_shops=int(num_shops)
             if target_area=="Seattle": target_area_bounds=(47.48, 47.73, -122.43, -122.22)
             else: print(f"Warn: No bounds for '{target_area}'. Using generic."); target_area_bounds=(0,1,0,1)
             print(f"Data loaded: {num_shops} shops. Area bounds:{target_area_bounds}")
             print("=== SciPy Setup (Diff Evolution) ==="); bounds=[];
             for _ in range(num_shops): bounds.extend([(target_area_bounds[2],target_area_bounds[3]), (target_area_bounds[0],target_area_bounds[1])])
             objective_func=lambda x: evaluate_facility_location(x, target_area_bounds)[0]
             print("--- Running SciPy DE ---")
             sci_result=differential_evolution(objective_func, bounds, maxiter=50, popsize=10, disp=False, seed=42)
             print("--- SciPy Results ---")
             if sci_result.success:
                 coords=sci_result.x; score=sci_result.fun; print(f"Success. Best Score(MinMaxDist):{score:.4f}"); locations_str_list = []
                 print("Locations(Lon,Lat):");
                 for i in range(num_shops): loc_str = f"Shop {i+1}:({coords[i*2]:.5f},{coords[i*2+1]:.5f})"; locations_str_list.append(loc_str); print(loc_str)
                 details = "; ".join(locations_str_list)
                 if min_dist is not None: details += f" [Note: Min dist constraint ({min_dist}) NOT enforced.]"; print(f"Note: Min dist constraint ({min_dist}) NOT enforced.")
                 result.update({"Status":"Success", "ObjectiveValue": score, "ResultDetails": details})
             else: result.update({"Status":"Failed - Optimization Error", "ResultDetails": f"SciPy DE failed: {sci_result.message}"}); print(f"Optimization failed: {sci_result.message}")

        elif problem_type == "NURSE_SCHEDULING_MGH":
            print("--- Porting NURSE_SCHEDULING_MGH (Simplified: Min Consecutive Penalty) ---")
            num_nurses=problem_data.get('num_nurses'); num_shifts=problem_data.get('num_shifts'); num_days=problem_data.get('num_days'); max_consecutive=problem_data.get('max_consecutive_days')
            if num_days is None or not isinstance(num_days,(int,float)) or num_days<=0: result.update({"Status":"Failed - Missing/Invalid Data", "ResultDetails":f"Invalid 'num_days'. Parsed:{num_days}. Check CSV."}); return result
            if not isinstance(num_nurses,(int,float)) or num_nurses<=0: result.update({"Status":"Failed - Invalid Data", "ResultDetails":"Invalid 'num_nurses'."}); return result
            if not isinstance(num_shifts,(int,float)) or num_shifts<=0: result.update({"Status":"Failed - Invalid Data", "ResultDetails":"Invalid 'num_shifts'."}); return result
            if not isinstance(max_consecutive,(int,float)) or max_consecutive<=0: result.update({"Status":"Failed - Invalid Data", "ResultDetails":"Invalid 'max_consecutive_days'."}); return result
            num_nurses,num_shifts,num_days,max_consecutive = int(num_nurses),int(num_shifts),int(num_days),int(max_consecutive)
            print(f"Data loaded: {num_nurses} nurses, {num_shifts} work shifts(+1 off), {num_days} days, max {max_consecutive} consecutive.")
            print("=== DEAP Setup (GA - Min Penalty) ===");
            if hasattr(creator, "FitnessMin"): del creator.FitnessMin
            if hasattr(creator, "Individual"): del creator.Individual
            creator.create("FitnessMin", base.Fitness, weights=(-1.0,)); creator.create("Individual", list, fitness=creator.FitnessMin)
            toolbox=base.Toolbox(); num_possible_shifts=num_shifts+1; toolbox.register("attr_shift", random.randrange, num_possible_shifts); ind_size=num_nurses*num_days; toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_shift, ind_size); toolbox.register("population", tools.initRepeat, list, toolbox.individual)
            toolbox.register("evaluate", evaluate_nurse_scheduling, num_nurses=num_nurses, num_days=num_days, max_consecutive=max_consecutive); toolbox.register("mate", tools.cxTwoPoint); toolbox.register("mutate", tools.mutUniformInt, low=0, up=num_shifts, indpb=0.02); toolbox.register("select", tools.selTournament, tournsize=3)
            print("--- Running DEAP GA ---"); POP_SIZE,MAX_GEN,CXPB,MUTPB = 60,70,0.7,0.3; pop=toolbox.population(n=POP_SIZE); hof=tools.HallOfFame(1);
            algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=MAX_GEN, halloffame=hof, verbose=False)
            print("--- DEAP Results (Simplified Nurse Scheduling) ---")
            if hof: best_ind=hof[0]; penalty=best_ind.fitness.values[0]; result.update({"Status":"Success", "ObjectiveValue":penalty, "ResultDetails":f"Best Schedule Penalty:{penalty:.1f}. Note: Only max consecutive days evaluated."}); print(f"Best Schedule Penalty:{penalty:.1f}(Lower=better)"); print("Note: Only max consecutive days evaluated.")
            else: result.update({"Status":"Success - No Solution Found"}); print("No solution found.")

        elif problem_type == "PORTFOLIO_OPTIMIZATION":
            print("--- Porting PORTFOLIO_OPTIMIZATION (Simplified: Min Variance) ---")
            assets=problem_data.get('list_of_potential_assets'); hist_perf=problem_data.get('historical_asset_performance_data'); corr_matrix=problem_data.get('asset_correlation_data'); target_return_level=0.10
            if not isinstance(assets, list): result.update({"Status":"Failed - Invalid Data", "ResultDetails":f"Invalid assets list. Type:{type(assets)}"}); return result
            num_assets = len(assets)
            data_status = "Placeholder Data"
            print(f"Warn [Idx{problem_index}]: Financial data processing NOT IMPLEMENTED. Using random placeholders.")
            if True: np.random.seed(42); expected_returns=np.random.rand(num_assets)*0.2+0.01; volatilities=np.random.rand(num_assets)*0.3+0.05; _t=np.random.rand(num_assets,num_assets); _c=(_t+_t.T)/2; np.fill_diagonal(_c, 1); D=np.diag(volatilities); cov_matrix=D@_c@D
            else: result.update({"Status":"Failed - Not Implemented", "ResultDetails":"Need to implement financial data processing."}); return result
            print(f"Data loaded: {num_assets} assets. Target Return:{target_return_level:.1%}")
            print("=== SciPy Setup (QP) ==="); objective=portfolio_variance; sum_to_one=LinearConstraint(np.ones(num_assets), 1.0, 1.0); ret_constr=LinearConstraint(expected_returns, target_return_level, np.inf); bounds=Bounds(0.0, 1.0); constraints=[sum_to_one, ret_constr]; init_w=np.ones(num_assets)/num_assets
            print("--- Running SciPy Optimizer (SLSQP) ---")
            sci_result=minimize(objective, init_w, args=(cov_matrix,), method='SLSQP', bounds=bounds, constraints=constraints, options={'disp': False, 'ftol': 1e-9})
            print("--- SciPy Results ---")
            if sci_result.success: weights=sci_result.x; ret=portfolio_return(weights, expected_returns); var=sci_result.fun; vol=np.sqrt(var); print(f"Success. Return={ret:.2%}, Volatility={vol:.2%}"); weights_str_list=[]; print("Weights:"); [ (weights_str_list.append(f"{asset}:{weights[i]:.2%}"), print(f"    - {asset}: {weights[i]:.2%}")) for i, asset in enumerate(assets) if weights[i]>1e-4] ; result.update({"Status":"Success", "ObjectiveValue":vol, "ObjectiveValue2":ret, "ResultDetails": f"Data: {data_status}. Weights: {'; '.join(weights_str_list)}"})
            else: result.update({"Status":"Failed - Optimization Error", "ResultDetails": f"SciPy SLSQP failed: {sci_result.message}"}); print(f"Optimization failed: {sci_result.message}")

        elif problem_type == "TIMETABLING_CONFERENCE":
            print("--- Porting TIMETABLING_CONFERENCE (Simplified: Min Clashes) ---")
            sessions_raw=problem_data.get('list_of_sessions_with_topics_speakers'); rooms_raw=problem_data.get('list_of_rooms_with_capacities'); slots_per_day=problem_data.get('timeslots_per_day'); num_days=problem_data.get('num_days')
            if not isinstance(sessions_raw, list): result.update({"Status":"Failed - Invalid Data", "ResultDetails":f"Invalid sessions list. Type:{type(sessions_raw)}" }); return result
            if not isinstance(rooms_raw, list): result.update({"Status":"Failed - Invalid Data", "ResultDetails":f"Invalid rooms list. Type:{type(rooms_raw)}" }); return result
            if not isinstance(slots_per_day, list): result.update({"Status":"Failed - Invalid Data", "ResultDetails":f"Invalid timeslots list. Type:{type(slots_per_day)}" }); return result
            if not isinstance(num_days,(int,float)) or num_days<=0: result.update({"Status":"Failed - Invalid Data", "ResultDetails":"Invalid num_days."}); return result
            num_days=int(num_days); num_slots_total=len(slots_per_day)*num_days; rooms={r['room_name']:r.get('capacity',float('inf')) for r in rooms_raw if 'room_name' in r}; num_rooms=len(rooms);
            if num_rooms==0: result.update({"Status":"Failed - Invalid Data", "ResultDetails":"No valid rooms."}); return result
            slots=list(range(num_slots_total)); avg_cap=sum(v for v in rooms.values() if isinstance(v,(int,float)))/num_rooms if num_rooms else 50
            sessions = [];
            for i, s in enumerate(sessions_raw):
                if isinstance(s, dict): s.setdefault('attendance', int(avg_cap/2)); sessions.append(s)
                else: print(f"Warn: Session data at index {i} not dict: {s}. Skipping.")
            num_sessions = len(sessions)
            if num_sessions == 0: result.update({"Status":"Failed - Invalid Data", "ResultDetails":"No valid sessions."}); return result
            print(f"Data loaded: {num_sessions} sessions, {num_rooms} rooms, {num_slots_total} slots.")
            print("=== DEAP Setup (GA - Min Penalty) ===");
            if hasattr(creator, "FitnessMin"): del creator.FitnessMin
            if hasattr(creator, "Individual"): del creator.Individual
            creator.create("FitnessMin", base.Fitness, weights=(-1.0,)); creator.create("Individual", list, fitness=creator.FitnessMin)
            toolbox=base.Toolbox(); toolbox.register("attr_slot", random.randrange, num_slots_total); toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_slot, num_sessions); toolbox.register("population", tools.initRepeat, list, toolbox.individual)
            toolbox.register("evaluate", evaluate_timetabling, sessions=sessions, rooms=rooms, slots=slots); toolbox.register("mate", tools.cxUniform, indpb=0.5); toolbox.register("mutate", tools.mutUniformInt, low=0, up=num_slots_total-1, indpb=0.05); toolbox.register("select", tools.selTournament, tournsize=3)
            print("--- Running DEAP GA ---"); POP_SIZE,MAX_GEN,CXPB,MUTPB = 60,80,0.7,0.3; pop=toolbox.population(n=POP_SIZE); hof=tools.HallOfFame(1);
            algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=MAX_GEN, halloffame=hof, verbose=False)
            print("--- DEAP Results (Simplified Timetabling) ---")
            if hof: best_ind=hof[0]; penalty=best_ind.fitness.values[0]; result.update({"Status":"Success", "ObjectiveValue":penalty, "ResultDetails":f"Best Schedule Penalty:{penalty:.1f}. Note: Basic constraints only."}); print(f"Best Schedule Penalty:{penalty:.1f}(Lower=better)"); print("Note: Only basic speaker/room/capa evaluated.")
            else: result.update({"Status":"Success - No Solution Found"}); print("No solution found.")

        elif problem_type == "PROJECT_SCHEDULING_CONSTRUCTION":
            print("--- Porting PROJECT_SCHEDULING_CONSTRUCTION (Simplified: Min Makespan) ---")
            tasks_raw=problem_data.get('list_of_tasks_with_durations_and_dependencies')
            if not isinstance(tasks_raw, list) or not tasks_raw: result.update({"Status":"Failed - Invalid Data", "ResultDetails":f"Invalid tasks list. Type:{type(tasks_raw)}" }); return result
            tasks=[]; valid_ids=set(); required_keys=['task_id','duration_days']
            try:
                 for task_dict in tasks_raw:
                      if not all(k in task_dict for k in required_keys): print(f"Warn: Skip task {task_dict.get('task_id','N/A')}"); continue
                      task_dict['duration_days']=float(task_dict['duration_days']); task_dict.setdefault('depends_on', [])
                      if task_dict['duration_days']<0: print(f"Warn: Task {task_dict['task_id']} neg duration."); continue
                      tasks.append(task_dict); valid_ids.add(task_dict['task_id'])
                 if not tasks: result.update({"Status":"Failed - Invalid Data", "ResultDetails":"No valid tasks."}); return result
                 num_tasks=len(tasks);
                 for task in tasks: task['depends_on']=[dep for dep in task['depends_on'] if dep in valid_ids]
            except Exception as e: result.update({"Status":"Failed - Data Processing Error", "ResultDetails":f"Error task data: {e}"}); return result
            print(f"Data loaded: {num_tasks} valid tasks.")
            print("=== DEAP Setup (GA - Min Makespan) ===");
            if hasattr(creator, "FitnessMin"): del creator.FitnessMin
            if hasattr(creator, "Individual"): del creator.Individual
            creator.create("FitnessMin", base.Fitness, weights=(-1.0,)); creator.create("Individual", list, fitness=creator.FitnessMin)
            toolbox=base.Toolbox(); toolbox.register("indices", random.sample, range(num_tasks), num_tasks); toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices); toolbox.register("population", tools.initRepeat, list, toolbox.individual)
            toolbox.register("evaluate", evaluate_project_scheduling, tasks=tasks); toolbox.register("mate", tools.cxOrdered); toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05); toolbox.register("select", tools.selTournament, tournsize=3)
            print("--- Running DEAP GA ---"); POP_SIZE,MAX_GEN,CXPB,MUTPB = 50,60,0.7,0.2; pop=toolbox.population(n=POP_SIZE); hof=tools.HallOfFame(1);
            algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=MAX_GEN, halloffame=hof, verbose=False)
            print("--- DEAP Results (Simplified Proj Sched) ---")
            if hof: best_ind=hof[0]; makespan=best_ind.fitness.values[0]; task_seq=[tasks[idx]['task_id'] for idx in best_ind]; seq_str = ' -> '.join(task_seq); result.update({"Status":"Success", "ObjectiveValue":makespan, "ResultDetails":f"Makespan:{makespan:.1f}days. Sequence: {seq_str}. Note: Resources NOT considered."}); print(f"Best Makespan:{makespan:.1f}days"); print(f"  Sequence: {seq_str}"); print("Note: Resources NOT considered.")
            else: result.update({"Status":"Success - No Solution Found"}); print("No solution found.")

        elif problem_type == "NETWORK_DESIGN_WATER":
            print("--- Porting NETWORK_DESIGN_WATER (Conceptual Placeholder) ---")
            num_homes=problem_data.get('num_homes'); pipe_types=problem_data.get('pipe_types_and_costs_per_unit_length_per_diameter')
            print("Warn: Water network design requires external data & simulation."); details = "Requires external data/simulation."
            if isinstance(pipe_types, list) and pipe_types:
                 print("Pipe Types(Ex):"); [print(f"  - {pt}") for pt in pipe_types[:3]]
                 print("\n=== Conceptual Setup ==="); print("Obj: Min pipe cost (conceptual)")
                 assumed_len=float(num_homes)*50 if num_homes and isinstance(num_homes, (int, float)) else 5000 # Added check for num_homes type
                 cheap_pipe=min(pipe_types, key=lambda x: x.get('cost_per_meter', float('inf')))
                 cost = cheap_pipe.get('cost_per_meter', 0) * assumed_len if isinstance(cheap_pipe.get('cost_per_meter'), (int, float)) else 0 # Added check for cost type
                 details = f"Conceptual min cost({assumed_len:.0f}m): ${cost:.2f}. Requires simulation."
                 print(f"Simplistic Min Cost({assumed_len:.0f}m): ${cost:.2f}"); print("Result: Cannot perform valid optimization.")
            else: print("Pipe data missing/invalid."); details += " Pipe data missing."
            print("--- Results: Skipped ---")
            result.update({"Status":"Skipped - Conceptual Only", "ResultDetails": details})

        else:
            details = f"Optimization logic for type '{problem_type}' is not implemented."
            print(details)
            result.update({"Status": "Failed - Not Implemented", "ResultDetails": details})

    except Exception as e:
        err_details = f"Runtime Error in run_optimization: {type(e).__name__} - {e}"
        print(err_details); # traceback.print_exc() # Uncomment for full traceback
        result.update({"Status": "Failed - Execution Error", "ResultDetails": err_details})

    return result


# --- Script Entry Point ---
if __name__ == "__main__":
    file_path = "problems_data_structured_manual.csv" # Make sure this matches your filename
    results_file_path = "optimization_results.csv"
    indices_to_run = None
    all_results = [] # List to store result dictionaries

    if len(sys.argv) > 1:
        try: indices_to_run = [int(arg) for arg in sys.argv[1:]]; print(f"Running only indices: {indices_to_run}")
        except ValueError: print("Warn: Args not integers. Running all."); indices_to_run = None

    if not os.path.exists(file_path): print(f"Error: File not found '{file_path}'"); sys.exit(1)
    print(f"Loading data from '{file_path}'...")
    try: df = pd.read_csv(file_path, dtype=str) # Read all as string initially
    except Exception as e: print(f"Error loading CSV '{file_path}': {e}"); sys.exit(1)
    if 'ProblemIndex' not in df.columns or 'DetectedType' not in df.columns: print(f"Error: Required columns missing."); sys.exit(1)

    try: df['ProblemIndex'] = df['ProblemIndex'].astype(int) # Convert index col to int
    except ValueError: print("Error: ProblemIndex column non-integer."); sys.exit(1)

    all_indices = sorted(df['ProblemIndex'].unique()); target_indices = indices_to_run if indices_to_run is not None else all_indices
    processed_count = 0

    for index in target_indices:
        if index not in all_indices: print(f"Warn: Index {index} not in file. Skipping."); continue
        print(f"\n{'='*20} Processing Problem Index: {index} {'='*20}")
        current_result = { # Default result
                "ProblemIndex": index, "DetectedType": "Unknown", "Status": "Failed - Parsing Error",
                "ObjectiveValue": None, "ObjectiveValue2": None, "ResultDetails": "Error before run_optimization call" }
        try:
            problem_data = parse_problem_data(df, index)
            current_result["DetectedType"] = problem_data.get('problem_type', 'Unknown')
            current_result["Status"] = "Processing"
            run_result = run_optimization(problem_data, index)
            current_result.update(run_result)
            processed_count += 1
        except ValueError as e: current_result.update({"Status": "Failed - Config Error", "ResultDetails": str(e)})
        except KeyError as e: current_result.update({"Status": "Failed - Missing Data Error", "ResultDetails": f"Key '{e}' not found"})
        except Exception as e:
             err_msg = f"Runtime Error before/during run_opt: {type(e).__name__} - {e}"
             print(err_msg); # traceback.print_exc()
             current_result.update({"Status": "Failed - Runtime Error", "ResultDetails": err_msg})
        finally:
            all_results.append(current_result)
            print("-" * 60)

    print(f"\n>>> Processing Complete <<<")
    print(f"Total problems attempted: {len(target_indices)}")
    print(f"Loop Iterations Completed: {processed_count}")
    status_counts = defaultdict(int) # Corrected loop
    for res in all_results:
        status_counts[res.get("Status", "Unknown Status")] += 1
    print("Final Status Counts:");
    for status, count in status_counts.items(): print(f"  - {status}: {count}")

    if all_results:
        try:
            print(f"\nSaving results to '{results_file_path}'...")
            df_results = pd.DataFrame(all_results)
            cols_order = ["ProblemIndex", "DetectedType", "Status", "ObjectiveValue", "ObjectiveValue2", "ResultDetails"]
            df_results = df_results[[col for col in cols_order if col in df_results.columns]]
            df_results.to_csv(results_file_path, index=False, encoding='utf-8')
            print("Results saved successfully.")
        except Exception as e: print(f"Error saving results to CSV: {e}")
    else: print("No results were generated to save.")

Warn: Args not integers. Running all.
Loading data from 'problems_data_structured_manual.csv'...

==================== Processing Problem Index: 0 ====================

--- Parsing Problem Index: 0, Type: TSP_FLIGHTS ---

>>> Processing Problem Index: 0 (TSP_FLIGHTS) <<<
--- Porting TSP_FLIGHTS ---
Data loaded: 10 cities.
=== DEAP Setup (Multi-Obj GA) ===
--- Running DEAP NSGA-II ---
--- DEAP Results ---
Non-dom solutions:9
 Sol 1 (Example): Amsterdam -> Berlin -> Budapest -> Paris -> London -> Madrid -> Barcelona -> Prague -> Vienna -> Rome -> Amsterdam | Fit(Cost,Dur):(2882.77261225, 69.04980647000001)
------------------------------------------------------------

==================== Processing Problem Index: 1 ====================

--- Parsing Problem Index: 1, Type: TSP_DRIVING_FUEL ---
DEBUG(Idx1): Parsing 'list_of_cities'. Post-strip type:<class 'str'>, val:["Yellowstone", "Yosemite", "Zion", "Olympic", "Glacier", "Acadia", "Teton", "Rocky Mountain"]
DEBUG(Idx1): Parsed type:<cla